In [1]:
from difflib import get_close_matches
import pandas as pd
from mobfot import MobFot

client  = MobFot()

def parse_time(time):
    return f"{time.replace('T', ' ').rsplit('.', 1)[0]}"


ids = list()
for x in client.get_league(77)['matches']['allMatches']:
    ids.append(x['id'])


df = pd.read_csv("FIFA DataSet/Data/FIFA World Cup 2022 Match Data/data.csv")
df['match_id'] = None

home_teams = df.home_team.unique()
away_teams = df.away_team.unique()
for id in ids:
    match = client.get_match_details(id)['general']
    parsed_time = parse_time(match['matchTimeUTCDate'])
    date_string = parsed_time.split(' ')[0]
    time_string = parsed_time.split(' ')[1]
    time_string = str((int(time_string.split(':')[0]) + 3) % 24) + ':00:00'
    time = date_string + ' ' + time_string
    home_team = match['homeTeam']['name']
    if home_team == 'USA':
        home_team = 'United States'
    if home_team == 'South Korea':
        home_team = 'Korea Republic'
    home_team = get_close_matches(home_team, home_teams, n=1, cutoff=0.5)[0]
    
    away_team = match['awayTeam']['name']
    if away_team == 'USA':
        away_team = 'United States'
    if away_team == 'South Korea':
        away_team = 'Korea Republic'
    away_team = get_close_matches(away_team, away_teams, n=1, cutoff=0.5)[0]
    print(f"Currently adding {time} {home_team} vs {away_team}")
    df.loc[(df.match_time == time) & (df.home_team == home_team) & (df.away_team == away_team), 'match_id'] = id

df.to_csv("FIFA DataSet/Data/FIFA World Cup 2022 Match Data/data.csv", index=False)


Currently adding 2022-11-20 19:00:00 Qatar vs Ecuador
Currently adding 2022-11-21 16:00:00 England vs IR Iran
Currently adding 2022-11-21 19:00:00 Senegal vs Netherlands
Currently adding 2022-11-21 22:00:00 United States vs Wales
Currently adding 2022-11-22 13:00:00 Argentina vs Saudi Arabia
Currently adding 2022-11-22 16:00:00 Denmark vs Tunisia
Currently adding 2022-11-22 19:00:00 Mexico vs Poland
Currently adding 2022-11-22 22:00:00 France vs Australia
Currently adding 2022-11-23 13:00:00 Morocco vs Croatia
Currently adding 2022-11-23 16:00:00 Germany vs Japan
Currently adding 2022-11-23 19:00:00 Spain vs Costa Rica
Currently adding 2022-11-23 22:00:00 Belgium vs Canada
Currently adding 2022-11-24 13:00:00 Switzerland vs Cameroon
Currently adding 2022-11-24 16:00:00 Uruguay vs Korea Republic
Currently adding 2022-11-24 19:00:00 Portugal vs Ghana
Currently adding 2022-11-24 22:00:00 Brazil vs Serbia
Currently adding 2022-11-25 13:00:00 Wales vs IR Iran
Currently adding 2022-11-25 16:

In [2]:
id = df.loc[0, 'match_id']
match = client.get_match_details(id)

In [3]:
match['content']['lineup']['lineup'][0]['players'][3][0]['id']

'613792'

In [6]:
from icecream import ic
import json

structure = match['content']['lineup']['lineup'][0]['lineup']
player_data = match['content']['lineup']['lineup'][0]['players'][3][0]
ic(player_data)

ic| player_data: {'events': {'as': 1, 'sub': {'subbedOut': 69}},
                  'fantasyScore': {'bgcolor': 'purple', 'num': None},
                  'id': '298448',


                  'imageUrl': 'https://images.fotmob.com/image_resources/playerimages/298448.png',
                  'isCaptain': False,
                  'isGoalkeeper': False,
                  'isHomeTeam': True,
                  'localizedPosition': {'key': 'centerattackingmidfielder_short', 'label': 'AM'},
                  'minutesPlayed': 69,
                  'name': {'firstName': 'Davy',
                           'fullName': 'Davy Klaassen',
                           'lastName': 'Klaassen'},
                  'pageUrl': '/players/298448/davy-klaassen',
                  'position': 'Midfielder',
                  'positionId': 95,
                  'positionRow': 4,
                  'positionStringShort': 'AM',
                  'rating': {'bgcolor': '#1ec853',
                             'isTop': {'isMatchFinished': True, 'isTopRating': False},
                             'num': '7.1'},
                  'role': 'Midfielder',
                  'shirt': 14,
            

{'id': '298448',
 'positionId': 95,
 'position': 'Midfielder',
 'positionStringShort': 'AM',
 'localizedPosition': {'label': 'AM',
  'key': 'centerattackingmidfielder_short'},
 'name': {'firstName': 'Davy',
  'lastName': 'Klaassen',
  'fullName': 'Davy Klaassen'},
 'shirt': 14,
 'usualPosition': 2,
 'usingOptaId': False,
 'teamId': '6708',
 'imageUrl': 'https://images.fotmob.com/image_resources/playerimages/298448.png',
 'pageUrl': '/players/298448/davy-klaassen',
 'isHomeTeam': True,
 'timeSubbedOn': None,
 'timeSubbedOff': 69,
 'positionRow': 4,
 'role': 'Midfielder',
 'isCaptain': False,
 'events': {'as': 1, 'sub': {'subbedOut': 69}},
 'rating': {'num': '7.1',
  'bgcolor': '#1ec853',
  'isTop': {'isTopRating': False, 'isMatchFinished': True}},
 'fantasyScore': {'num': None, 'bgcolor': 'purple'},
 'minutesPlayed': 69,
 'shotmap': [],
 'stats': [{'title': 'Top stats',
   'key': 'top_stats',
   'stats': {'FotMob rating': {'key': 'rating_title', 'value': 7.1},
    'Minutes played': {'ke

In [84]:
homeTeam = match['general']['homeTeam']['name']
awayTeam = match['general']['awayTeam']['name']

homeTeam

'Qatar'

In [81]:
client.get_team('613792')

In [20]:


def get_player_data(match):
    homePlayers = list()
    for x in match['content']['lineup']['lineup'][0]['players']:
        for y in x:
            homePlayers.append(y)

    awayPlayers = list()
    for x in match['content']['lineup']['lineup'][1]['players']:
        for y in x:
            awayPlayers.append(y)
            
    return homePlayers, awayPlayers
            

def get_player_by_name(name, players):
    allPlayers = [x['name']['lastName'] for x in players]
    try:

        match = get_close_matches(name, allPlayers, n=1, cutoff=0.5)[0]
        return players[allPlayers.index(match)]
    except:
        return None
    


id = 3854556
match = client.get_match_details(id)

homePlayers, awayPlayers = get_player_data(match)

ic(get_player_by_name('Van Dijk', homePlayers))

ic| get_player_by_name('Van Dijk', homePlayers): {'events': {},
                                                  'fantasyScore': {'bgcolor': 'purple', 'num': None},
                                                  'id': '209405',
                                                  'imageUrl': 'https://images.fotmob.com/image_resources/playerimages/209405.png',
                                                  'isCaptain': True,
                                                  'isGoalkeeper': False,
                                                  'isHomeTeam': True,
                                                  'localizedPosition': {'key': 'centerback_short', 'label': 'CB'},
                                                  'minutesPlayed': 90,
                                                  'name': {'firstName': 'Virgil',
                                                           'fullName': 'Virgil van Dijk',
                                                           'lastNam

{'id': '209405',
 'positionId': 35,
 'position': 'Defender',
 'positionStringShort': 'CB',
 'localizedPosition': {'label': 'CB', 'key': 'centerback_short'},
 'name': {'firstName': 'Virgil',
  'lastName': 'van Dijk',
  'fullName': 'Virgil van Dijk'},
 'shirt': 4,
 'usualPosition': 1,
 'usingOptaId': False,
 'teamId': '6708',
 'imageUrl': 'https://images.fotmob.com/image_resources/playerimages/209405.png',
 'pageUrl': '/players/209405/virgil-van-dijk',
 'isHomeTeam': True,
 'timeSubbedOn': None,
 'timeSubbedOff': None,
 'positionRow': 1,
 'role': 'Defender',
 'isCaptain': True,
 'events': {},
 'rating': {'num': '7.1',
  'bgcolor': '#1ec853',
  'isTop': {'isTopRating': False, 'isMatchFinished': True}},
 'fantasyScore': {'num': None, 'bgcolor': 'purple'},
 'minutesPlayed': 90,
 'shotmap': [],
 'stats': [{'title': 'Top stats',
   'key': 'top_stats',
   'stats': {'FotMob rating': {'key': 'rating_title', 'value': 7.05},
    'Minutes played': {'key': 'minutes_played', 'value': 90},
    'Goals'

In [87]:

player_data = match['content']['lineup']['lineup'][0]['players'][3][1]
player_data

{'id': '644017',
 'positionId': 106,
 'position': 'Attacker',
 'positionStringShort': 'CF',
 'localizedPosition': {'label': 'CF', 'key': 'centerforward_short'},
 'name': {'firstName': 'Almoez', 'lastName': 'Ali', 'fullName': 'Almoez Ali'},
 'shirt': 19,
 'usualPosition': 3,
 'usingOptaId': False,
 'teamId': '5902',
 'imageUrl': 'https://images.fotmob.com/image_resources/playerimages/644017.png',
 'pageUrl': '/players/644017/almoez-ali',
 'isHomeTeam': True,
 'timeSubbedOn': None,
 'timeSubbedOff': 72,
 'positionRow': 5,
 'role': 'Attacker',
 'isCaptain': False,
 'events': {'yc': 1, 'sub': {'subbedOut': 72}},
 'rating': {'num': '5.6',
  'bgcolor': '#f08022',
  'isTop': {'isTopRating': False, 'isMatchFinished': True}},
 'fantasyScore': {'num': None, 'bgcolor': 'purple'},
 'minutesPlayed': 72,
 'shotmap': [{'id': 2486977597,
   'eventType': 'Miss',
   'teamId': 5902,
   'playerId': 644017,
   'playerName': 'Almoez Ali',
   'x': 99.01754385950001,
   'y': 29.295476189000002,
   'min': 45,


In [22]:
match

{'general': {'matchId': '3854556',
  'matchName': 'Netherlands-vs-Ecuador_Fri, Nov 25, 2022, 16:00 UTC',
  'matchRound': '2',
  'teamColors': {'darkMode': {'home': '#FB6132', 'away': '#ECD743'},
   'lightMode': {'home': '#FB6132', 'away': '#e29b00'},
   'fontDarkMode': {'home': 'rgba(255, 255, 255, 1.0)',
    'away': 'rgba(29, 29, 29, 1.0)'},
   'fontLightMode': {'home': 'rgba(255, 255, 255, 1.0)',
    'away': 'rgba(255, 255, 255, 1.0)'}},
  'leagueId': 868712,
  'leagueName': 'World Cup Grp. A',
  'leagueRoundName': 'Round 2',
  'parentLeagueId': 77,
  'countryCode': 'INT',
  'parentLeagueName': 'World Cup',
  'parentLeagueSeason': '2022',
  'parentLeagueTopScorerLink': 'https://data.fotmob.com/stats/77/season/15338/goals.json',
  'parentLeagueTournamentId': 15338,
  'homeTeam': {'name': 'Netherlands', 'id': 6708},
  'awayTeam': {'name': 'Ecuador', 'id': 6707},
  'coverageLevel': 'xG',
  'matchTimeUTC': 'Fri, Nov 25, 2022, 16:00 UTC',
  'matchTimeUTCDate': '2022-11-25T16:00:00.000Z',
